[View in Colaboratory](https://colab.research.google.com/github/heumchri/ESPNet/blob/master/binaryESPNet.ipynb)

# general

In [0]:
#!kill -9 -1

In [0]:
ls /content

## utilization monitoring

In [0]:
#requirements for gpu and ram usage

# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize


In [0]:
#gpu and ram usage

import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [0]:
#running processes

#!ps -aux
!ps -aux | grep python

In [0]:
#storage usage

!df -h 

## increase shm for multithreading to work

In [0]:
cd /etc/

In [0]:
%%writefile fstab
tmpfs /dev/shm tmpfs defaults,size=2G 0 0

In [0]:
!mount -o remount /dev/shm

In [0]:
#storage usage

!df -h 

# mount google drive

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
#!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#!apt-get update -qq 2>&1 > /dev/null
#!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
cd /content/

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
ls

# Install ESPNet

In [0]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

In [0]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

In [0]:
cd /content/

In [0]:
!git clone https://github.com/heumchri/ESPNet.git

In [0]:
cd /content/ESPNet/

In [0]:
!git pull

# test forward time
20 classes:

decoder batch 1: 0.045

decoder batch 16: 0.027

encoder batch 1: 0.034

encoder batch 16: 0.020

binary: 

decoder batch 1: 0.038

decoder batch 16: 0.022

encoder batch 1: 0.034

encoder batch 16: 0.020

In [0]:
cd /content/ESPNet/test/

In [0]:
!python3 eval_forwardTime_pretrained.py --batch-size 1 --modelType 1 --classes 20

In [0]:
!python3 eval_forwardTime.py --batch-size 1 --modelType 1 --classes 2 --weightsDir /content/drive/espnet_checkpoints/

# test pretrained model (all classes, val dataset) 0.952

In [0]:
cd /content/ESPNet/test/

In [0]:
!python3 VisualizeResults.py --modelType 1 --data_dir /content/datasets/cityscapes/leftImg8bit/val/frankfurt/
!python3 VisualizeResults.py --modelType 1 --data_dir /content/datasets/cityscapes/leftImg8bit/val/lindau/
!python3 VisualizeResults.py --modelType 1 --data_dir /content/datasets/cityscapes/leftImg8bit/val/munster/

## visualize results

In [0]:
#ls results/

In [0]:
from IPython.display import Image
#Image('/content/datasets/cityscapes/leftImg8bit/val/munster/munster_000000_000019_leftImg8bit.png')


In [0]:
#Image('/content/datasets/cityscapes/gtFine/val/munster/munster_000000_000019_gtFine_color.png')

In [0]:
#Image('results/c_munster_000000_000019_leftImg8bit.png')

In [0]:
Image('results/over_munster_000000_000019_leftImg8bit.jpg')

In [0]:
#Image('results/munster_000000_000019_leftImg8bit.png')

## evaluate results

In [0]:
import os
os.environ['CITYSCAPES_RESULTS'] = '/content/ESPNet/test/results/'
os.environ['CITYSCAPES_DATASET'] = '/content/datasets/cityscapes/'

In [0]:
cd /content/cityscapesScripts/cityscapesscripts/evaluation/

In [0]:
!python2 evalPixelLevelSemanticLabeling.py 

# test pretrained model (binary, val dataset) 0.952

In [0]:
cd /content/ESPNet/test/

In [0]:
!python3 VisualizeResults_pretrained_binary.py --modelType 1 --data_dir /content/datasets/cityscapes/leftImg8bit/val/frankfurt/
!python3 VisualizeResults_pretrained_binary.py --modelType 1 --data_dir /content/datasets/cityscapes/leftImg8bit/val/lindau/
!python3 VisualizeResults_pretrained_binary.py --modelType 1 --data_dir /content/datasets/cityscapes/leftImg8bit/val/munster/

## visualize results

In [0]:
#ls results/

In [0]:
from IPython.display import Image
#Image('/content/datasets/cityscapes/leftImg8bit/val/munster/munster_000000_000019_leftImg8bit.png')


In [0]:
#Image('/content/datasets/cityscapes/gtFine/val/munster/munster_000000_000019_gtFine_color.png')

In [0]:
#Image('results/c_munster_000000_000019_leftImg8bit.png')

In [0]:
Image('results/over_munster_000000_000019_leftImg8bit.jpg')

In [0]:
#Image('results/munster_000000_000019_leftImg8bit.png')

## evaluate results

In [0]:
import os
os.environ['CITYSCAPES_RESULTS'] = '/content/ESPNet/test/results/'
os.environ['CITYSCAPES_DATASET'] = '/content/datasets/cityscapes/'

In [0]:
cd /content/cityscapesScripts/cityscapesscripts/evaluation/

In [0]:
!python2 evalPixelLevelSemanticLabeling.py 

# test pretrained model (all classes, val dataset, only encoder) 0.926

In [0]:
cd /content/ESPNet/test/

In [0]:
!python3 VisualizeResults.py --modelType 2 --data_dir /content/datasets/cityscapes/leftImg8bit/val/frankfurt/
!python3 VisualizeResults.py --modelType 2 --data_dir /content/datasets/cityscapes/leftImg8bit/val/lindau/
!python3 VisualizeResults.py --modelType 2 --data_dir /content/datasets/cityscapes/leftImg8bit/val/munster/

## visualize results

In [0]:
#ls results/

In [0]:
from IPython.display import Image
#Image('/content/datasets/cityscapes/leftImg8bit/val/munster/munster_000000_000019_leftImg8bit.png')


In [0]:
#Image('/content/datasets/cityscapes/gtFine/val/munster/munster_000000_000019_gtFine_color.png')

In [0]:
#Image('results/c_munster_000000_000019_leftImg8bit.png')

In [0]:
Image('results/over_munster_000000_000019_leftImg8bit.jpg')

In [0]:
#Image('results/munster_000000_000019_leftImg8bit.png')

## evaluate results

In [0]:
import os
os.environ['CITYSCAPES_RESULTS'] = '/content/ESPNet/test/results/'
os.environ['CITYSCAPES_DATASET'] = '/content/datasets/cityscapes/'

In [0]:
cd /content/cityscapesScripts/cityscapesscripts/evaluation/

In [0]:
!python2 evalPixelLevelSemanticLabeling.py 

# test retrained model (binary, val dataset, only encoder) 0.950
**epoch 16 (= model_17.pth, best val loss) : 0.944**

**epoch 6 (best road val Acc) : 0.934**

**epoch 9 (best val mAcc) : 0.938 **

**epoch 112 : 0.947**

**epoch 220 (best road val IoU, best val mIoU) : 0.950**

In [0]:
cd /content/ESPNet/test/

In [0]:
!python3 VisualizeResults_retrained_binary.py --modelType 2 --data_dir /content/datasets/cityscapes/leftImg8bit/val/frankfurt/ --classes 2 --weightsDir /content/drive/espnet_checkpoints/
!python3 VisualizeResults_retrained_binary.py --modelType 2 --data_dir /content/datasets/cityscapes/leftImg8bit/val/lindau/ --classes 2 --weightsDir /content/drive/espnet_checkpoints/
!python3 VisualizeResults_retrained_binary.py --modelType 2 --data_dir /content/datasets/cityscapes/leftImg8bit/val/munster/ --classes 2 --weightsDir /content/drive/espnet_checkpoints/

## visualize results

In [0]:
#ls results/

In [0]:
from IPython.display import Image
#Image('/content/datasets/cityscapes/leftImg8bit/val/munster/munster_000000_000019_leftImg8bit.png')


In [0]:
#Image('/content/datasets/cityscapes/gtFine/val/munster/munster_000000_000019_gtFine_color.png')

In [0]:
#Image('results/c_munster_000000_000019_leftImg8bit.png')

In [0]:
Image('results/over_munster_000000_000019_leftImg8bit.jpg')

In [0]:
#Image('results/munster_000000_000019_leftImg8bit.png')

## evaluate results

In [0]:
import os
os.environ['CITYSCAPES_RESULTS'] = '/content/ESPNet/test/results/'
os.environ['CITYSCAPES_DATASET'] = '/content/datasets/cityscapes/'

In [0]:
cd /content/cityscapesScripts/cityscapesscripts/evaluation/

In [0]:
!python2 evalPixelLevelSemanticLabeling.py 

# test retrained model (binary, val dataset) 0.958
**epoch 156: 0.952**

**epoch 227: 0.958**

In [0]:
cd /content/ESPNet/test/

In [0]:
!python3 VisualizeResults_retrained_binary.py --modelType 1 --data_dir /content/datasets/cityscapes/leftImg8bit/val/frankfurt/ --classes 2 --weightsDir /content/drive/espnet_checkpoints/
!python3 VisualizeResults_retrained_binary.py --modelType 1 --data_dir /content/datasets/cityscapes/leftImg8bit/val/lindau/ --classes 2 --weightsDir /content/drive/espnet_checkpoints/
!python3 VisualizeResults_retrained_binary.py --modelType 1 --data_dir /content/datasets/cityscapes/leftImg8bit/val/munster/ --classes 2 --weightsDir /content/drive/espnet_checkpoints/

## visualize results

In [0]:
#ls results/

In [0]:
from IPython.display import Image
#Image('/content/datasets/cityscapes/leftImg8bit/val/munster/munster_000000_000019_leftImg8bit.png')


In [0]:
#Image('/content/datasets/cityscapes/gtFine/val/munster/munster_000000_000019_gtFine_color.png')

In [0]:
#Image('results/c_munster_000000_000019_leftImg8bit.png')

In [0]:
Image('results/over_munster_000000_000019_leftImg8bit.jpg')

In [0]:
#Image('results/munster_000000_000019_leftImg8bit.png')

## evaluate results

In [0]:
import os
os.environ['CITYSCAPES_RESULTS'] = '/content/ESPNet/test/results/'
os.environ['CITYSCAPES_DATASET'] = '/content/datasets/cityscapes/'

In [0]:
cd /content/cityscapesScripts/cityscapesscripts/evaluation/

In [0]:
!python2 evalPixelLevelSemanticLabeling.py 

# qualitative test retrained model (binary)

##cityscapes video

In [0]:
cd /content/datasets/cityscapes/

In [0]:
!wget --keep-session-cookies --save-cookies=cookies.txt --post-data 'username=heumann.christopher&password=xxx&submit=Login' https://www.cityscapes-dataset.com/login/

In [0]:
!wget --load-cookies cookies.txt --content-disposition https://www.cityscapes-dataset.com/file-handling/?packageID=12

In [0]:
!unzip leftImg8bit_demoVideo.zip

In [0]:
!rm cookies.txt index.html README leftImg8bit_demoVideo.zip

In [0]:
cd /content/ESPNet/test/

In [0]:
!python3 VisualizeResults_retrained_binary.py --modelType 1 --data_dir /content/datasets/cityscapes/leftImg8bit/demoVideo/stuttgart_00/ --classes 2 --weightsDir /content/ESPNet/retrained_binary/
!python3 VisualizeResults_retrained_binary.py --modelType 1 --data_dir /content/datasets/cityscapes/leftImg8bit/demoVideo/stuttgart_01/ --classes 2 --weightsDir /content/ESPNet/retrained_binary/
!python3 VisualizeResults_retrained_binary.py --modelType 1 --data_dir /content/datasets/cityscapes/leftImg8bit/demoVideo/stuttgart_02/ --classes 2 --weightsDir /content/ESPNet/retrained_binary/

### visualize results

In [0]:
ls results/

In [0]:
from IPython.display import Image

In [0]:
Image('results/over_stuttgart_00_000000_000001_leftImg8bit.jpg')

In [0]:
!apt-get install ffmpeg

In [0]:
!ffmpeg -framerate 24 -pattern_type glob -i 'results/over_stuttgart_00*.jpg' -c:v libx264 -pix_fmt yuv420p over_stuttgart_00.mp4

In [0]:
!ffmpeg -framerate 24 -pattern_type glob -i 'results/over_stuttgart_01*.jpg' -c:v libx264 -pix_fmt yuv420p over_stuttgart_01.mp4

In [0]:
!ffmpeg -framerate 24 -pattern_type glob -i 'results/over_stuttgart_02*.jpg' -c:v libx264 -pix_fmt yuv420p over_stuttgart_02.mp4

In [0]:
from google.colab import files
files.download('over_stuttgart_00.mp4')
files.download('over_stuttgart_01.mp4')
files.download('over_stuttgart_02.mp4')

##apex video

In [0]:
!mkdir -p /content/datasets/apex

In [0]:
%%capture
!unzip /content/drive/apex/images/1.zip -d /content/datasets/apex/

In [0]:
cd /content/datasets/apex/1

In [0]:
!rename 's/\d+/sprintf("%05d",$&)/e' frame*

In [0]:
cd /content/ESPNet/test/

### input bilinear to 1024x512, output bilinear to 1368x1096

In [0]:
args_model="ESPNet"#, help='Model name')
args_data_dir="/content/datasets/apex/1/"#, help='Data directory')
args_img_extn="jpg"#, help='RGB Image format')
args_inWidth=1024#, help='Width of RGB image')
args_inHeight=512#, help='Height of RGB image')
args_scaleIn=1#, help='For ESPNet-C, scaleIn=8. For ESPNet, scaleIn=1')
args_modelType=1#, help='1=ESPNet, 2=ESPNet-C')
args_savedir='./results1'#, help='directory to save the results')
args_gpu=True#, help='Run on CPU or GPU. If TRUE, then GPU.')
args_decoder=True#,help='True if ESPNet. False for ESPNet-C')  # False for encoder
args_weightsDir='/content/ESPNet/retrained_binary/'#, help='Pretrained weights directory.')
args_p=2#, help='depth multiplier. Supported only 2')
args_q=8#, help='depth multiplier. Supported only 3, 5, 8')
args_cityFormat=True#, help='If you want to convert to cityscape '
                                                                       #'original label ids')
args_colored=True#, type=bool, help='If you want to visualize the '
                                                                   #'segmentation masks in color')
args_overlay=True#, type=bool, help='If you want to visualize the '
                                                                   #'segmentation masks overlayed on top of RGB image')
args_classes=2#, type=int, help='Number of classes in the dataset. 20 for Cityscapes')

assert args_modelType == 1 and args_decoder#, 'Model type should be 2 for ESPNet-C and 1 for ESPNet'
if args_overlay:
    args_colored = True # This has to be true if you want to overlay

import numpy as np
import torch
from torch.autograd import Variable
import glob
import cv2
from PIL import Image as PILImage
import Model as Net
import os
import time
from argparse import ArgumentParser

pallete = [[0, 255, 0],
           [0, 0, 0]]


def relabel(img):
    '''
    This function relabels the predicted labels so that cityscape dataset can process
    :param img:
    :return:
    '''
    img[img == 0] = 7
    img[img == 1] = 0
    return img


def evaluateModel_resizeInput_resizeOutput(model, up, image_list):
    # gloabl mean and std values
    mean = [72.3923111, 82.90893555, 73.15840149]
    std = [45.3192215, 46.15289307, 44.91483307]

    for i, imgName in enumerate(image_list):
        img = cv2.imread(imgName)
        if args_overlay:
            img_orig = np.copy(img)

        img = img.astype(np.float32)
        for j in range(3):
            img[:, :, j] -= mean[j]
        for j in range(3):
            img[:, :, j] /= std[j]

        # resize the image to 1024x512x3
        img = cv2.resize(img, (1024, 512))

        img /= 255
        img = img.transpose((2, 0, 1))
        img_tensor = torch.from_numpy(img)
        img_tensor = torch.unsqueeze(img_tensor, 0)  # add a batch dimension
        img_variable = Variable(img_tensor, volatile=True)
        if args_gpu:
            img_variable = img_variable.cuda()
        img_out = model(img_variable)

        img_out = up(img_out)

        classMap_numpy = img_out[0].max(0)[1].byte().cpu().data.numpy()

        if i % 100 == 0:
            print(i)

        name = imgName.split('/')[-1]

        if args_colored:
            classMap_numpy_color = np.zeros((img_orig.shape[0], img_orig.shape[1], img_orig.shape[2]), dtype=np.uint8)
            for idx in range(len(pallete)):
                [r, g, b] = pallete[idx]
                classMap_numpy_color[classMap_numpy == idx] = [b, g, r]
            cv2.imwrite(args_savedir + os.sep + 'c_' + name.replace(args_img_extn, 'png'), classMap_numpy_color)
            if args_overlay:
                output_grey = cv2.cvtColor(classMap_numpy_color,cv2.COLOR_BGR2GRAY)
                ret, mask = cv2.threshold(output_grey, 0, 255, cv2.THRESH_BINARY_INV)
                orig_masked = cv2.bitwise_or(img_orig, img_orig, mask=mask)
                orig_colored = dst = cv2.add(classMap_numpy_color,orig_masked)
                overlayed = cv2.addWeighted(orig_colored, 0.5, img_orig, 0.5, 0)
                cv2.imwrite(args_savedir + os.sep + 'over_' + name.replace(args_img_extn, 'jpg'), overlayed)

        if args_cityFormat:
            classMap_numpy = relabel(classMap_numpy.astype(np.uint8))

        cv2.imwrite(args_savedir + os.sep + name.replace(args_img_extn, 'png'), classMap_numpy)


def main_resizeInput_resizeOutput():
    # read all the images in the folder
    image_list = glob.glob(args_data_dir + os.sep + '*.' + args_img_extn)

    up = None
    if args_modelType == 2:
        up = torch.nn.Upsample(scale_factor=16, mode='bilinear')
    else:
        up = torch.nn.Upsample(size=(1096,1368), mode='bilinear')
    if args_gpu:
        up = up.cuda()

    p = args_p
    q = args_q
    classes = args_classes
    if args_modelType == 2:
        modelA = Net.ESPNet_Encoder(classes, p, q)  # Net.Mobile_SegNetDilatedIA_C_stage1(20)
        model_weight_file = args_weightsDir + os.sep + 'encoder' + os.sep + 'espnet_p_' + str(p) + '_q_' + str(
            q) + '.pth'
        if not os.path.isfile(model_weight_file):
            print('Pre-trained model file does not exist. Please check ../pretrained/encoder folder')
            exit(-1)
        modelA.load_state_dict(torch.load(model_weight_file))
    elif args_modelType == 1:
        modelA = Net.ESPNet(classes, p, q)  # Net.Mobile_SegNetDilatedIA_C_stage1(20)
        model_weight_file = args_weightsDir + os.sep + 'decoder' + os.sep + 'espnet_p_' + str(p) + '_q_' + str(q) + '.pth'
        if not os.path.isfile(model_weight_file):
            print('Pre-trained model file does not exist. Please check ../pretrained/decoder folder')
            exit(-1)
        modelA.load_state_dict(torch.load(model_weight_file))
    else:
        print('Model not supported')
    # modelA = torch.nn.DataParallel(modelA)
    if args_gpu:
        modelA = modelA.cuda()

    # set to evaluation mode
    modelA.eval()

    if not os.path.isdir(args_savedir):
        os.mkdir(args_savedir)

    evaluateModel_resizeInput_resizeOutput(modelA, up, image_list)

In [0]:
main_resizeInput_resizeOutput()

### input unchanged, output unchanged

In [0]:
args_model="ESPNet"#, help='Model name')
args_data_dir="/content/datasets/apex/1/"#, help='Data directory')
args_img_extn="jpg"#, help='RGB Image format')
args_inWidth=1024#, help='Width of RGB image')
args_inHeight=512#, help='Height of RGB image')
args_scaleIn=1#, help='For ESPNet-C, scaleIn=8. For ESPNet, scaleIn=1')
args_modelType=1#, help='1=ESPNet, 2=ESPNet-C')
args_savedir='./results1'#, help='directory to save the results')
args_gpu=True#, help='Run on CPU or GPU. If TRUE, then GPU.')
args_decoder=True#,help='True if ESPNet. False for ESPNet-C')  # False for encoder
args_weightsDir='/content/ESPNet/retrained_binary/'#, help='Pretrained weights directory.')
args_p=2#, help='depth multiplier. Supported only 2')
args_q=8#, help='depth multiplier. Supported only 3, 5, 8')
args_cityFormat=True#, help='If you want to convert to cityscape '
                                                                       #'original label ids')
args_colored=True#, type=bool, help='If you want to visualize the '
                                                                   #'segmentation masks in color')
args_overlay=True#, type=bool, help='If you want to visualize the '
                                                                   #'segmentation masks overlayed on top of RGB image')
args_classes=2#, type=int, help='Number of classes in the dataset. 20 for Cityscapes')

assert args_modelType == 1 and args_decoder#, 'Model type should be 2 for ESPNet-C and 1 for ESPNet'
if args_overlay:
    args_colored = True # This has to be true if you want to overlay

import numpy as np
import torch
from torch.autograd import Variable
import glob
import cv2
from PIL import Image as PILImage
import Model as Net
import os
import time
from argparse import ArgumentParser

pallete = [[0, 255, 0],
           [0, 0, 0]]


def relabel(img):
    '''
    This function relabels the predicted labels so that cityscape dataset can process
    :param img:
    :return:
    '''
    img[img == 0] = 7
    img[img == 1] = 0
    return img


def evaluateModel(model, up, image_list):
    # gloabl mean and std values
    mean = [72.3923111, 82.90893555, 73.15840149]
    std = [45.3192215, 46.15289307, 44.91483307]

    for i, imgName in enumerate(image_list):
        img = cv2.imread(imgName)
        if args_overlay:
            img_orig = np.copy(img)

        img = img.astype(np.float32)
        for j in range(3):
            img[:, :, j] -= mean[j]
        for j in range(3):
            img[:, :, j] /= std[j]

        # resize the image to 1024x512x3
        #img = cv2.resize(img, (1024, 512))

        img /= 255
        img = img.transpose((2, 0, 1))
        img_tensor = torch.from_numpy(img)
        img_tensor = torch.unsqueeze(img_tensor, 0)  # add a batch dimension
        img_variable = Variable(img_tensor, volatile=True)
        if args_gpu:
            img_variable = img_variable.cuda()
        img_out = model(img_variable)

        #img_out = up(img_out)

        classMap_numpy = img_out[0].max(0)[1].byte().cpu().data.numpy()

        if i % 100 == 0:
            print(i)

        name = imgName.split('/')[-1]

        if args_colored:
            classMap_numpy_color = np.zeros((img_orig.shape[0], img_orig.shape[1], img_orig.shape[2]), dtype=np.uint8)
            for idx in range(len(pallete)):
                [r, g, b] = pallete[idx]
                classMap_numpy_color[classMap_numpy == idx] = [b, g, r]
            cv2.imwrite(args_savedir + os.sep + 'c_' + name.replace(args_img_extn, 'png'), classMap_numpy_color)
            if args_overlay:
                output_grey = cv2.cvtColor(classMap_numpy_color,cv2.COLOR_BGR2GRAY)
                ret, mask = cv2.threshold(output_grey, 0, 255, cv2.THRESH_BINARY_INV)
                orig_masked = cv2.bitwise_or(img_orig, img_orig, mask=mask)
                orig_colored = dst = cv2.add(classMap_numpy_color,orig_masked)
                overlayed = cv2.addWeighted(orig_colored, 0.5, img_orig, 0.5, 0)
                cv2.imwrite(args_savedir + os.sep + 'over_' + name.replace(args_img_extn, 'jpg'), overlayed)

        if args_cityFormat:
            classMap_numpy = relabel(classMap_numpy.astype(np.uint8))

        cv2.imwrite(args_savedir + os.sep + name.replace(args_img_extn, 'png'), classMap_numpy)


def main():
    # read all the images in the folder
    image_list = glob.glob(args_data_dir + os.sep + '*.' + args_img_extn)

    up = None
    if args_modelType == 2:
        up = torch.nn.Upsample(scale_factor=16, mode='bilinear')
    else:
        up = torch.nn.Upsample(size=(1096,1368), mode='bilinear')
    if args_gpu:
        up = up.cuda()

    p = args_p
    q = args_q
    classes = args_classes
    if args_modelType == 2:
        modelA = Net.ESPNet_Encoder(classes, p, q)  # Net.Mobile_SegNetDilatedIA_C_stage1(20)
        model_weight_file = args_weightsDir + os.sep + 'encoder' + os.sep + 'espnet_p_' + str(p) + '_q_' + str(
            q) + '.pth'
        if not os.path.isfile(model_weight_file):
            print('Pre-trained model file does not exist. Please check ../pretrained/encoder folder')
            exit(-1)
        modelA.load_state_dict(torch.load(model_weight_file))
    elif args_modelType == 1:
        modelA = Net.ESPNet(classes, p, q)  # Net.Mobile_SegNetDilatedIA_C_stage1(20)
        model_weight_file = args_weightsDir + os.sep + 'decoder' + os.sep + 'espnet_p_' + str(p) + '_q_' + str(q) + '.pth'
        if not os.path.isfile(model_weight_file):
            print('Pre-trained model file does not exist. Please check ../pretrained/decoder folder')
            exit(-1)
        modelA.load_state_dict(torch.load(model_weight_file))
    else:
        print('Model not supported')
    # modelA = torch.nn.DataParallel(modelA)
    if args_gpu:
        modelA = modelA.cuda()

    # set to evaluation mode
    modelA.eval()

    if not os.path.isdir(args_savedir):
        os.mkdir(args_savedir)

    evaluateModel(modelA, up, image_list)

In [0]:
main()

### input bilinear to 1024x512, output unchanged

In [0]:
args_model="ESPNet"#, help='Model name')
args_data_dir="/content/datasets/apex/1/"#, help='Data directory')
args_img_extn="jpg"#, help='RGB Image format')
args_inWidth=1024#, help='Width of RGB image')
args_inHeight=512#, help='Height of RGB image')
args_scaleIn=1#, help='For ESPNet-C, scaleIn=8. For ESPNet, scaleIn=1')
args_modelType=1#, help='1=ESPNet, 2=ESPNet-C')
args_savedir='./results1'#, help='directory to save the results')
args_gpu=True#, help='Run on CPU or GPU. If TRUE, then GPU.')
args_decoder=True#,help='True if ESPNet. False for ESPNet-C')  # False for encoder
args_weightsDir='/content/ESPNet/retrained_binary/'#, help='Pretrained weights directory.')
args_p=2#, help='depth multiplier. Supported only 2')
args_q=8#, help='depth multiplier. Supported only 3, 5, 8')
args_cityFormat=True#, help='If you want to convert to cityscape '
                                                                       #'original label ids')
args_colored=True#, type=bool, help='If you want to visualize the '
                                                                   #'segmentation masks in color')
args_overlay=True#, type=bool, help='If you want to visualize the '
                                                                   #'segmentation masks overlayed on top of RGB image')
args_classes=2#, type=int, help='Number of classes in the dataset. 20 for Cityscapes')

assert args_modelType == 1 and args_decoder#, 'Model type should be 2 for ESPNet-C and 1 for ESPNet'
if args_overlay:
    args_colored = True # This has to be true if you want to overlay

import numpy as np
import torch
from torch.autograd import Variable
import glob
import cv2
from PIL import Image as PILImage
import Model as Net
import os
import time
from argparse import ArgumentParser

pallete = [[0, 255, 0],
           [0, 0, 0]]


def relabel(img):
    '''
    This function relabels the predicted labels so that cityscape dataset can process
    :param img:
    :return:
    '''
    img[img == 0] = 7
    img[img == 1] = 0
    return img


def evaluateModel_resizeInput(model, up, image_list):
    # gloabl mean and std values
    mean = [72.3923111, 82.90893555, 73.15840149]
    std = [45.3192215, 46.15289307, 44.91483307]

    for i, imgName in enumerate(image_list):
        img = cv2.imread(imgName)
        if args_overlay:
            img_orig = np.copy(img)
            img_orig = cv2.resize(img, (1024, 512))

        img = img.astype(np.float32)
        for j in range(3):
            img[:, :, j] -= mean[j]
        for j in range(3):
            img[:, :, j] /= std[j]

        # resize the image to 1024x512x3
        img = cv2.resize(img, (1024, 512))

        img /= 255
        img = img.transpose((2, 0, 1))
        img_tensor = torch.from_numpy(img)
        img_tensor = torch.unsqueeze(img_tensor, 0)  # add a batch dimension
        img_variable = Variable(img_tensor, volatile=True)
        if args_gpu:
            img_variable = img_variable.cuda()
        img_out = model(img_variable)

        classMap_numpy = img_out[0].max(0)[1].byte().cpu().data.numpy()

        if i % 100 == 0:
            print(i)

        name = imgName.split('/')[-1]

        if args_colored:
            classMap_numpy_color = np.zeros((img_orig.shape[0], img_orig.shape[1], img_orig.shape[2]), dtype=np.uint8)
            for idx in range(len(pallete)):
                [r, g, b] = pallete[idx]
                classMap_numpy_color[classMap_numpy == idx] = [b, g, r]
            cv2.imwrite(args_savedir + os.sep + 'c_' + name.replace(args_img_extn, 'png'), classMap_numpy_color)
            if args_overlay:
                output_grey = cv2.cvtColor(classMap_numpy_color,cv2.COLOR_BGR2GRAY)
                ret, mask = cv2.threshold(output_grey, 0, 255, cv2.THRESH_BINARY_INV)
                orig_masked = cv2.bitwise_or(img_orig, img_orig, mask=mask)
                orig_colored = dst = cv2.add(classMap_numpy_color,orig_masked)
                overlayed = cv2.addWeighted(orig_colored, 0.5, img_orig, 0.5, 0)
                cv2.imwrite(args_savedir + os.sep + 'over_' + name.replace(args_img_extn, 'jpg'), overlayed)

        if args_cityFormat:
            classMap_numpy = relabel(classMap_numpy.astype(np.uint8))

        cv2.imwrite(args_savedir + os.sep + name.replace(args_img_extn, 'png'), classMap_numpy)


def main_resizeInput():
    # read all the images in the folder
    image_list = glob.glob(args_data_dir + os.sep + '*.' + args_img_extn)

    up = None
    if args_modelType == 2:
        up = torch.nn.Upsample(scale_factor=16, mode='bilinear')
    else:
        up = torch.nn.Upsample(size=(1096,1368), mode='bilinear')
    if args_gpu:
        up = up.cuda()

    p = args_p
    q = args_q
    classes = args_classes
    if args_modelType == 2:
        modelA = Net.ESPNet_Encoder(classes, p, q)  # Net.Mobile_SegNetDilatedIA_C_stage1(20)
        model_weight_file = args_weightsDir + os.sep + 'encoder' + os.sep + 'espnet_p_' + str(p) + '_q_' + str(
            q) + '.pth'
        if not os.path.isfile(model_weight_file):
            print('Pre-trained model file does not exist. Please check ../pretrained/encoder folder')
            exit(-1)
        modelA.load_state_dict(torch.load(model_weight_file))
    elif args_modelType == 1:
        modelA = Net.ESPNet(classes, p, q)  # Net.Mobile_SegNetDilatedIA_C_stage1(20)
        model_weight_file = args_weightsDir + os.sep + 'decoder' + os.sep + 'espnet_p_' + str(p) + '_q_' + str(q) + '.pth'
        if not os.path.isfile(model_weight_file):
            print('Pre-trained model file does not exist. Please check ../pretrained/decoder folder')
            exit(-1)
        modelA.load_state_dict(torch.load(model_weight_file))
    else:
        print('Model not supported')
    # modelA = torch.nn.DataParallel(modelA)
    if args_gpu:
        modelA = modelA.cuda()

    # set to evaluation mode
    modelA.eval()

    if not os.path.isdir(args_savedir):
        os.mkdir(args_savedir)

    evaluateModel_resizeInput(modelA, up, image_list)

In [0]:
main_resizeInput()

### visualize results

In [0]:
ls ./results1/

In [0]:
from IPython.display import Image
Image("./results1/over_frame00000.jpg")

In [0]:
!apt-get install ffmpeg

In [0]:
!ffmpeg -framerate 20 -pattern_type glob -i 'results1/over_*.jpg' -c:v libx264 -pix_fmt yuv420p over_bag1.mp4

In [0]:
from google.colab import files
files.download('over_bag1.mp4')

In [0]:
ls 

In [0]:
!mv over_bag1.mp4 /content/drive/apex/

# retrain model (binary, cityscapes)

In [0]:
!pip install graphviz

In [0]:
cp /content/ESPNet/train/city/* /content/datasets/cityscapes/

### train on small set (20 train, 20 val)

In [0]:
#cd /content/datasets/cityscapes/

In [0]:
#rm -r train.txt val.txt

In [0]:
#mv train_small.txt train.txt

In [0]:
#mv val_small.txt val.txt

##retrain encoder

### start training

In [0]:
cd /content/ESPNet/train/

In [0]:
!python3 main.py --scaleIn 8 --data_dir /content/datasets/cityscapes/ --savedir /content/drive/espnet_checkpoints/encoder/ --classes 2 --batch_size 14 --max_epochs 300 --num_workers 4

###resume training

In [0]:
cd /content/ESPNet/train/

In [0]:
!python3 main.py --scaleIn 8 --data_dir /content/datasets/cityscapes/ --savedir /content/drive/espnet_checkpoints/encoder/ --classes 2 --batch_size 14 --max_epochs 300 --num_workers 4 --resume True --resumeLoc /content/drive/espnet_checkpoints/encoder/_enc_2_8/checkpoint.pth.tar 

##retrain decoder

### start training

In [0]:
cd /content/ESPNet/train/

In [0]:
!python3 main.py --scaleIn 1 --decoder True --pretrained /content/drive/espnet_checkpoints/encoder/epoch219_950_espnet_p_2_q_8.pth --data_dir /content/datasets/cityscapes/ --savedir /content/drive/espnet_checkpoints/decoder/ --classes 2 --batch_size 12 --max_epochs 300 --num_workers 4

###resume training

In [0]:
cd /content/ESPNet/train/

In [0]:
!python3 main.py --scaleIn 1 --decoder True --pretrained /content/drive/espnet_checkpoints/encoder/epoch219_950_espnet_p_2_q_8.pth --data_dir /content/datasets/cityscapes/ --savedir /content/drive/espnet_checkpoints/decoder/ --classes 2 --batch_size 12 --max_epochs 300 --num_workers 4 --resume True --resumeLoc /content/drive/espnet_checkpoints/decoder/_dec_2_8/checkpoint.pth.tar 

# retrain model (binary, apex)

In [0]:
!pip install graphviz

In [0]:
!mkdir -p /content/datasets/apex

In [0]:
%%capture
!unzip /content/drive/apex/annotated/apex_annotated.zip -d /content/datasets/apex/

In [0]:
import PIL.Image as Image
import numpy as np
import os

def relabel():
  i = 1
  for imgpath in os.listdir(os.getcwd()):
    labelOrig = Image.open(imgpath)
    labelOrig = labelOrig.convert(mode = "L")
    labelNumpy = np.array(labelOrig)
    labelNumpy[labelNumpy==142] = 1
    im2 = Image.fromarray(labelNumpy)
    im2.save(imgpath)
    if i%10 == 0:
      print(i)
    i = i+1

In [0]:
cd /content/datasets/apex/train/labels/

In [0]:
relabel()

In [0]:
cd /content/datasets/apex/val/labels/

In [0]:
relabel()

##retrain encoder

### start training

In [0]:
cd /content/ESPNet/train/

In [0]:
!python3 main_apex.py --scaleIn 8 --data_dir /content/datasets/apex/ --savedir /content/drive/apex_espnet_checkpoints/encoder/ --classes 2 --batch_size 4 --max_epochs 300 --num_workers 4 --inWidth 1368 --inHeight 1096 

###resume training

In [0]:
cd /content/ESPNet/train/

In [0]:
!python3 main_apex.py --scaleIn 8 --data_dir /content/datasets/apex/ --savedir /content/drive/apex_espnet_checkpoints/encoder/ --classes 2 --batch_size 4 --max_epochs 300 --num_workers 4 --resume True --resumeLoc /content/drive/apex_espnet_checkpoints/encoder/_enc_2_8/checkpoint.pth.tar --inWidth 1368 --inHeight 1096 

##retrain decoder

### start training

In [0]:
cd /content/ESPNet/train/

In [0]:
!python3 main_apex.py --scaleIn 1 --decoder True --pretrained /content/drive/apex_espnet_checkpoints/encoder/_enc_2_8/model_261.pth --data_dir /content/datasets/apex/ --savedir /content/drive/apex_espnet_checkpoints/decoder/ --classes 2 --batch_size 4 --max_epochs 300 --num_workers 4 --inWidth 1368 --inHeight 1096 

###resume training

In [0]:
cd /content/ESPNet/train/

In [0]:
!python3 main_apex.py --scaleIn 1 --decoder True --pretrained /content/drive/apex_espnet_checkpoints/encoder/_enc_2_8/model_261.pth --data_dir /content/datasets/apex/ --savedir /content/drive/apex_espnet_checkpoints/decoder/ --classes 2 --batch_size 4 --max_epochs 300 --num_workers 4 --resume True --resumeLoc /content/drive/apex_espnet_checkpoints/decoder/_dec_2_8/checkpoint.pth.tar --inWidth 1368 --inHeight 1096  

# retrain model (binary, apex_test)

In [0]:
!pip install graphviz

In [0]:
!mkdir -p /content/datasets/apex

In [0]:
%%capture
!unzip /content/drive/apex/annotated/apex_annotated_test.zip -d /content/datasets/apex/

In [0]:
import PIL.Image as Image
import numpy as np
import os

def relabel():
  i = 1
  for imgpath in os.listdir(os.getcwd()):
    labelOrig = Image.open(imgpath)
    labelOrig = labelOrig.convert(mode = "L")
    labelNumpy = np.array(labelOrig)
    labelNumpy[labelNumpy==142] = 1
    im2 = Image.fromarray(labelNumpy)
    im2.save(imgpath)
    if i%10 == 0:
      print(i)
    i = i+1

In [0]:
cd /content/datasets/apex/train/labels/

In [0]:
relabel()

In [0]:
cd /content/datasets/apex/val/labels/

In [0]:
relabel()

##retrain encoder

### start training

In [0]:
cd /content/ESPNet/train/

In [0]:
!python3 main_apex.py --scaleIn 8 --data_dir /content/datasets/apex/ --savedir /content/drive/apex_test_espnet_checkpoints/encoder/ --classes 2 --batch_size 4 --max_epochs 300 --num_workers 4 --inWidth 1368 --inHeight 1096 

[57/62] loss: 0.046 time:1.29
[58/62] loss: 0.026 time:1.30
[59/62] loss: 0.020 time:1.28
[60/62] loss: 0.036 time:1.29


###resume training

In [0]:
cd /content/ESPNet/train/

In [0]:
!python3 main_apex.py --scaleIn 8 --data_dir /content/datasets/apex/ --savedir /content/drive/apex_test_espnet_checkpoints/encoder/ --classes 2 --batch_size 4 --max_epochs 300 --num_workers 4 --resume True --resumeLoc /content/drive/apex_test_espnet_checkpoints/encoder/_enc_2_8/checkpoint.pth.tar --inWidth 1368 --inHeight 1096 

##retrain decoder

### start training

In [0]:
cd /content/ESPNet/train/

In [0]:
!python3 main_apex.py --scaleIn 1 --decoder True --pretrained /content/drive/apex_test_espnet_checkpoints/encoder/_enc_2_8/model_261.pth --data_dir /content/datasets/apex/ --savedir /content/drive/apex_test_espnet_checkpoints/decoder/ --classes 2 --batch_size 4 --max_epochs 300 --num_workers 4 --inWidth 1368 --inHeight 1096 

###resume training

In [0]:
cd /content/ESPNet/train/

In [0]:
!python3 main_apex.py --scaleIn 1 --decoder True --pretrained /content/drive/apex_test_espnet_checkpoints/encoder/_enc_2_8/model_261.pth --data_dir /content/datasets/apex/ --savedir /content/drive/apex_test_espnet_checkpoints/decoder/ --classes 2 --batch_size 4 --max_epochs 300 --num_workers 4 --resume True --resumeLoc /content/drive/apex_test_espnet_checkpoints/decoder/_dec_2_8/checkpoint.pth.tar --inWidth 1368 --inHeight 1096  